In [1]:
import json
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
ratings_df = pd.read_csv("./dataset/ratings.csv")
ratings_df = ratings_df.drop("timestamp", 1)
ratings_df = ratings_df.fillna(0)
ratings_df

C:\Users\mattg_3roa89k\AppData\Local\Temp\ipykernel_23752\3584907209.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop("timestamp", 1)


,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
...,...,...,...
20000258,138493,68954,4.5
20000259,138493,69526,4.5
20000260,138493,69644,3.0
20000261,138493,70286,5.0


In [3]:
ratings_df.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [4]:
ratings_df_length = len(ratings_df.index)

In [5]:
def dictToJSONFile(filePath, dictionary):
    with open(filePath, 'w') as fp:
        json.dump(dictionary, fp, indent=2)

In [6]:
isRatingsMatrixSaved = True

In [7]:
ratings_matrix = None
movie_ids = {} # key: movieId, value: movie_id index in user_movie_ratings
user_ids = {} # key: userId, value: movie_id index in user_movie_ratings
ratings_matrix = None

if not isRatingsMatrixSaved:
    # 138493 distinct users, 26744 distinct movies (from EDA notebook)
    ratings_matrix = np.zeros((138493, 26744), dtype=np.float32)

    for i, row in ratings_df.iterrows():
        user_id = row["userId"]
        movie_id = row["movieId"]
        rating = row["rating"]

        if not user_id in user_ids:
            user_ids[user_id] = len(user_ids)

        if not movie_id in movie_ids:
            movie_ids[movie_id] = len(movie_ids)

        ratings_matrix[user_ids[user_id], movie_ids[movie_id]] = rating

        print("{0} / {1}".format(i, ratings_df_length), end = "\r")

    np.save('ratings_matrix.npy', ratings_matrix) # save
    dictToJSONFile("movie_ids.json", movie_ids)
    dictToJSONFile("user_ids.json", user_ids)
else:
    ratings_matrix = np.load('ratings_matrix.npy') # load

    # movie_ids.json will used a str float when generated above
    # use vs code replace all function on .0 to make it just the int id
    with open('movie_ids.json', 'r') as fp:
        movie_ids = json.load(fp)

    with open('user_ids.json', 'r') as fp:
        user_ids = json.load(fp)

In [ ]:
movie_ids_list = [x for _, x in sorted(zip(list(movie_ids.values()), list(map(int, map(float, movie_ids.keys())))))]

dictToJSONFile("./movie_ids_list.json", movie_ids_list)

In [8]:
K = 10
ratings = ratings_matrix[:10000, :5000]
R = ratings

In [9]:
R.shape

(10000, 5000)

In [10]:
R

array([[3.5, 3.5, 3.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 4. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 2.5, ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

In [11]:
R.dtype

dtype('float32')

In [12]:
N = len(ratings)
M = len(ratings[0])
P = np.random.rand(N, K)
Q = np.random.rand(M, K)
ratings = tf.placeholder(tf.float32, name="ratings")
P_matrix = tf.Variable(P, dtype=tf.float32)
Q_matrix = tf.Variable(Q, dtype=tf.float32)
P_times_Q = tf.matmul(P_matrix, Q_matrix, transpose_b=True)
squared_error = tf.square(P_times_Q - ratings)
loss = tf.reduce_sum(squared_error)

In [13]:
P_times_Q

<tf.Tensor 'MatMul:0' shape=(10000, 5000) dtype=float32>

In [14]:
optimizer = tf.train.GradientDescentOptimizer(0.000001) # had to decrease the learning rate significantly
train = optimizer.minimize(loss)

In [15]:
# gpu_options = tf.GPUOptions(allow_growth=True)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

for i in range(5000):
    sess.run(train, {ratings: R})
    print("{0} / {1}".format(i+1, 5000), end = "\r")

saver.save(sess, './models/model_1')

'./models/model_1'

In [16]:
final_P_times_Q = np.around(sess.run(P_times_Q), 3)
print(final_P_times_Q)
print()

[[ 0.547  0.395  1.351 ...  0.01   0.025  0.007]
 [ 0.338  0.121  0.728 ... -0.007  0.015 -0.008]
 [ 0.29   0.752  1.774 ...  0.095  0.126  0.118]
 ...
 [ 0.205  0.235  1.121 ... -0.004  0.003 -0.025]
 [ 0.488  0.048  1.286 ... -0.044 -0.024 -0.043]
 [ 1.438 -0.108  1.542 ... -0.039 -0.021  0.011]]



In [17]:
final_P_matrix = np.around(sess.run(P_matrix), 3)
print(final_P_matrix)
print()

[[ 0.218  0.197  0.516 ...  0.337  0.093  0.169]
 [ 0.252  0.043  0.048 ...  0.206 -0.001  0.059]
 [ 0.199  0.346  0.845 ...  0.637  0.316  0.705]
 ...
 [ 0.153  0.392 -0.062 ...  0.312 -0.071  0.229]
 [ 0.673 -0.086  0.148 ...  0.482  0.448  0.041]
 [ 0.495  0.33  -0.235 ...  0.195  0.61   0.493]]



In [20]:
final_Q_matrix = np.transpose(np.around(sess.run(Q_matrix), 3))
dictToJSONFile("q-matrix.json", final_Q_matrix.tolist())
print(final_Q_matrix)
print()

[[ 0.63  -0.154  1.172 ... -0.056 -0.097 -0.077]
 [ 0.237  0.079  0.633 ...  0.006 -0.034 -0.011]
 [-0.091  0.356  0.147 ...  0.072  0.088  0.091]
 ...
 [ 0.19   0.139  0.52  ... -0.033  0.014 -0.035]
 [ 0.394 -0.034 -0.029 ...  0.029  0.077  0.097]
 [-0.049  0.19   0.765 ...  0.038 -0.004  0.043]]



In [21]:
new_user_indices = [1, 2, 4, 5]
new_user_ratings = [2, 2, 1, 2]

In [22]:
new_user_P_row_initial = np.random.rand(1, K)
new_user_P_row = tf.Variable(new_user_P_row_initial, dtype=tf.float32)
new_user_P_row_times_Q = tf.matmul(new_user_P_row, final_Q_matrix)
res = tf.gather(new_user_P_row_times_Q, new_user_indices, axis=1)
squared_error = tf.square(new_user_ratings - res)
loss = tf.reduce_sum(squared_error)
optimizer = tf.train.GradientDescentOptimizer(0.01)
predict = optimizer.minimize(loss)

In [24]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(50000):
    sess.run(predict)
    print("{0} / {1}".format(i+1, 50000), end = "\r")

In [25]:
final_new_user_P_row_times_Q = np.around(sess.run(new_user_P_row_times_Q), 3)
print(np.round(final_new_user_P_row_times_Q))

[[3. 2. 2. ... 0. 1. 1.]]
